In [1]:
import pandas as pd 
import numpy as np 
import warnings
warnings.filterwarnings("ignore")

In [7]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running. This can also be happening if non-NVIDIA GPU is running as primary display, and NVIDIA GPU is in WDDM mode.



In [8]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17527135962130252495
xla_global_id: -1
]


In [3]:
import tensorflow as tf
print("Available GPUs:", tf.config.list_physical_devices('GPU'))

Available GPUs: []


In [5]:
import tensorflow as tf
print("Available GPUs:", tf.config.list_physical_devices('GPU'))


Available GPUs: []


In [3]:
data = pd.read_csv("IMDB Dataset.csv")
data

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [4]:
data.shape

(50000, 2)

In [5]:
type(data)

pandas.core.frame.DataFrame

In [6]:
data['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [7]:
data.replace({'sentiment':{"positive":1,"negative":0}},inplace=True)

In [8]:
data

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0


In [9]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [10]:
train_data,test_data = train_test_split(data,test_size=0.2,random_state=42)

In [13]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


**Data Preprocessing**

In [19]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['review'])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']),maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']),maxlen=200)

In [20]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [21]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [22]:
y_train = train_data['sentiment']
y_test = test_data['sentiment']

In [28]:
y_train

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64

In [29]:
y_test

33553    1
9427     1
199      0
12447    1
39489    0
        ..
28567    0
25079    1
18707    1
15200    0
5857     1
Name: sentiment, Length: 10000, dtype: int64

**Building The LSTM Model**

In [25]:
# Build the model

model = Sequential()
model.add(Embedding(input_dim=5000,output_dim=128, input_length=200))
model.add(LSTM(128,dropout=0.2, recurrent_dropout=0.2)) 
model.add(Dense(1,activation='sigmoid'))

In [26]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 200, 128)          640000    
                                                                 
 lstm_2 (LSTM)               (None, 128)               131584    
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 771,713
Trainable params: 771,713
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.compile(
    optimizer='adam', 
    loss = 'binary_crossentropy', 
    metrics='accuracy'
)

**Training The Model**

In [31]:
model.fit(X_train,y_train,epochs=5,batch_size=50,validation_split=0.2)

Epoch 1/5
640/640 [==============================] - 446s 675ms/step - loss: 0.4575 - accuracy: 0.7847 - val_loss: 0.3461 - val_accuracy: 0.8510
Epoch 2/5
640/640 [==============================] - 386s 604ms/step - loss: 0.2913 - accuracy: 0.8823 - val_loss: 0.3002 - val_accuracy: 0.8766
Epoch 3/5
640/640 [==============================] - 386s 603ms/step - loss: 0.2267 - accuracy: 0.9116 - val_loss: 0.3287 - val_accuracy: 0.8585
Epoch 4/5
640/640 [==============================] - 385s 602ms/step - loss: 0.1937 - accuracy: 0.9256 - val_loss: 0.2937 - val_accuracy: 0.8821
Epoch 5/5
640/640 [==============================] - 383s 599ms/step - loss: 0.1614 - accuracy: 0.9385 - val_loss: 0.3171 - val_accuracy: 0.8806


**Model Evaluation**

In [32]:
loss, accuracy = model.evaluate(X_test,y_test)
print(f"Test Loss:{loss}")
print(f"Test Accuracy:{accuracy}")

313/313 [==============================] - 24s 77ms/step - loss: 0.3102 - accuracy: 0.8833
Test Loss:0.31020858883857727
Test Accuracy:0.8833000063896179


**Building The Predictive System**

In [60]:
def predict_sentiment(review):
    # tokenize and pad the review data
    sequence = tokenizer.texts_to_sequences(review)
    padded_sequence = pad_sequences(sequence, maxlen=200)
    prediction = model.predict(padded_sequence)
    sentiment = "Positive" if prediction[0][0] > 0.5 else "Negative"
    print(prediction[0][0])
    return sentiment
    

In [61]:
predict_sentiment("One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare. Forget pretty pictures painted for mainstream audiences, forget charm, forget romance...OZ doesn't mess around. The first episode I ever saw struck me as so nasty it was surreal, I couldn't say I was ready for it, but as I watched more, I developed a taste for Oz, and got accustomed to the high levels of graphic violence. Not just violence, but injustice (crooked guards who'll be sold out for a nickel, inmates who'll kill on order and get away with it, well mannered, middle class inmates being turned into prison bitches due to their lack of street skills or prison experience) Watching Oz, you may become comfortable with what is uncomfortable viewing....thats if you can get in touch with your darker side.")

56/56 [==============================] - 5s 80ms/step
0.7022398


'Positive'